In [1]:
import os
import PIL
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import h5py
import cv2
%matplotlib inline

In [2]:
labels = []

In [3]:
filename = None

for filename in range(1, 3065):
    with h5py.File('./brain_tumor_dataset/imageData/{}.mat'.format(filename), 'r') as f:
        label = f['cjdata']['label'][0][0] - 1
        labels.append(int(label))

In [4]:
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Dense, Conv2D , MaxPooling2D, Flatten
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator

In [5]:
inputs = Input(shape = (64, 64, 1))
conv1 =  Conv2D(32, (3,3), activation = 'relu', input_shape = [64, 64, 1])(inputs)
pool1 = MaxPooling2D(2)(conv1)
conv2 = Conv2D(32,(3,3), activation = 'relu')(pool1)
pool2 = MaxPooling2D(2)(conv2)
flatten = Flatten()(pool2)
dense1 = Dense(64, activation = 'relu')(flatten)
outputs = Dense(3, activation = 'softmax')(dense1)
model = Model(inputs=inputs, outputs=outputs)


model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 64, 64, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 62, 62, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 29, 29, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 6272)              0         
_________________________________________________________________
dense (Dense)                (None, 64)                401472

In [6]:
for i in range(len(labels)):
    if(labels[i]==0):
        labels[i] = "glioma"
    elif(labels[i]==1):
        labels[i] = "meningioma"
    else:
        labels[i] = "pituitary"
        

In [7]:
def importing_data(path):
    sample = []
    for filename in os.listdir(path):
        if "jpg" in filename:
            sample.append(filename)
    return sample

In [8]:
path = './brain_tumor_dataset/bt_images/'
train = importing_data(path)

train_data = pd.DataFrame({'image':train, 'label': labels})

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_val = train_test_split(train_data, test_size = 0.1, shuffle = True, random_state = 42)

In [10]:
IMG_SIZE = 64
size = (IMG_SIZE,IMG_SIZE)

In [ ]:
datagen = ImageDataGenerator(rescale = 1./255)

train_set = datagen.flow_from_dataframe(X_train,
                                        directory = './brain_tumor_dataset/bt_images/',
                                        x_col = 'image',
                                        y_col = 'label',
                                        target_size = size,
                                        color_mode = 'grayscale',
                                        class_mode = 'categorical',
                                        batch_size = 10,
                                        shuffle = True,
                                        interpolation = 'bilinear')

val_set = datagen.flow_from_dataframe(X_val,
                                      directory = './brain_tumor_dataset/bt_images/',
                                      x_col = 'image',
                                      y_col = 'label',
                                      target_size = size,
                                      color_mode = 'grayscale',
                                      class_mode = 'categorical',
                                      batch_size = 10,
                                      shuffle = True,
                                      interpolation = 'bilinear')

    

model.compile(optimizer = "Adam", loss='categorical_crossentropy', metrics =['acc'])  
es = EarlyStopping(monitor='val_loss', mode='min', patience=2, restore_best_weights=True, verbose=1)

checkpoint_cb = ModelCheckpoint("./models/abiwinanda.h5", save_best_only=True)

reduce_lr = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.2, patience = 3, min_lr = 1e-5, mode = 'min', verbose=1)

history = model.fit(train_set, validation_data = val_set, epochs= 50, batch_size = 10, callbacks=[es, checkpoint_cb, reduce_lr])   

In [11]:
def preprocess(img):
    img = img.astype(np.float32)
    img /= 255.0
    img = np.expand_dims(img,0)
    return img

In [12]:
val = X_val["image"].values.tolist()

In [31]:
class_names = {0: "glioma", 1: "meningoma", 2:"pituitary"}
d = 3
iters = 600
popsize = 10
model_path = "./models/abiwinanda.h5"

In [14]:
import keras 
keras_model = keras.models.load_model(model_path)
keras_model.compile(optimizer = "Adam", loss='categorical_crossentropy', metrics =['acc'])

In [15]:
def perturb(x):
    adv_img = orig.copy()

    pixs = np.array(np.split(x, len(x)/3)).astype(int)
    loc = (pixs[:, 0], pixs[:,1])
    val = pixs[:, 2]
    adv_img[loc] = val
    adv_img = np.expand_dims(adv_img,0)
    return adv_img

In [16]:
def optimize(x):
    adv_img = perturb(x)

    prob = keras_model.predict(preprocess(adv_img[0]))
    pred_orig = np.argmax(prob_orig)

    return prob[0][pred_orig]

In [17]:
pred_adv = 0
prob_adv = 0
ct = 0
def callback(x, convergence):
    global pred_adv, prob_adv, ct
    adv_img = perturb(x)


    prob = keras_model.predict(preprocess(adv_img[0]))

    pred_adv = np.argmax(prob)
    prob_adv = prob[0][pred_adv]
    if pred_adv != pred_orig:
        ct = ct+1
        return True

In [21]:
from scipy.optimize import differential_evolution

for filename in val:
    pred_adv = 0
    prob_adv = 0
    orig = cv2.imread(os.path.join("./brain_tumor_dataset/bt_images/",filename))
    orig = cv2.cvtColor(orig, cv2.COLOR_BGR2GRAY)
    orig = cv2.resize(orig, size)
    
    img = orig.copy()
    img = img.reshape(64,64,1)
    img = np.expand_dims(img,0)
    shape = img.shape
    
    
    prob_orig = keras_model.predict(preprocess(img[0]))
    pred_orig = np.argmax(prob_orig)

    bounds = [(0, shape[1]-1), (0, shape[2]-1), (0, 255)] * d
    result = differential_evolution(optimize, bounds, maxiter=iters, popsize=popsize, tol=1e-5, callback=callback)

    adv_img = perturb(result.x)

    prob = keras_model.predict(preprocess(adv_img[0]))
    print('Prob [%s]: %f --> Prob[%s]: %f ' %(class_names[pred_orig], prob_orig[0][pred_orig], class_names[pred_adv], prob_adv))

    if(pred_orig != pred_adv):
        cv2.imwrite('./outputs/'+ filename, adv_img[0])

Prob [pituitary]: 0.993369 --> Prob[pituitary]: 0.943707 
Prob [glioma]: 0.953879 --> Prob[glioma]: 0.634042 
Prob [pituitary]: 0.997594 --> Prob[pituitary]: 0.904879 
Prob [pituitary]: 0.997377 --> Prob[pituitary]: 0.984622 
Prob [meningoma]: 0.546422 --> Prob[pituitary]: 0.738574 
Prob [glioma]: 0.999829 --> Prob[glioma]: 0.997784 
Prob [meningoma]: 0.999976 --> Prob[meningoma]: 0.999959 
Prob [pituitary]: 0.959425 --> Prob[pituitary]: 0.663671 
Prob [glioma]: 0.999224 --> Prob[glioma]: 0.997691 
Prob [pituitary]: 0.991699 --> Prob[pituitary]: 0.932326 
Prob [meningoma]: 0.940403 --> Prob[meningoma]: 0.667465 
Prob [meningoma]: 0.999993 --> Prob[meningoma]: 0.999981 
Prob [glioma]: 0.999685 --> Prob[glioma]: 0.997041 
Prob [pituitary]: 0.992432 --> Prob[pituitary]: 0.683080 
Prob [pituitary]: 0.996604 --> Prob[pituitary]: 0.986183 
Prob [meningoma]: 0.999250 --> Prob[meningoma]: 0.986398 
Prob [meningoma]: 0.999862 --> Prob[meningoma]: 0.998439 
Prob [meningoma]: 0.999999 --> Prob[me

Prob [meningoma]: 0.999757 --> Prob[meningoma]: 0.998621 
Prob [pituitary]: 0.927992 --> Prob[glioma]: 0.512390 
Prob [meningoma]: 0.982540 --> Prob[meningoma]: 0.869043 
Prob [meningoma]: 0.523526 --> Prob[pituitary]: 0.706950 
Prob [meningoma]: 1.000000 --> Prob[meningoma]: 1.000000 
Prob [meningoma]: 0.985665 --> Prob[meningoma]: 0.905643 
Prob [pituitary]: 0.968168 --> Prob[pituitary]: 0.669924 
Prob [pituitary]: 0.966583 --> Prob[pituitary]: 0.722290 
Prob [pituitary]: 0.996189 --> Prob[pituitary]: 0.934092 
Prob [pituitary]: 0.728881 --> Prob[meningoma]: 0.670612 
Prob [glioma]: 0.997844 --> Prob[glioma]: 0.975274 
Prob [meningoma]: 0.984902 --> Prob[meningoma]: 0.704732 
Prob [glioma]: 0.884958 --> Prob[glioma]: 0.513919 
Prob [pituitary]: 0.996393 --> Prob[pituitary]: 0.972155 
Prob [meningoma]: 0.999291 --> Prob[meningoma]: 0.994595 
Prob [glioma]: 0.962434 --> Prob[glioma]: 0.875845 
Prob [pituitary]: 0.997432 --> Prob[pituitary]: 0.986130 
Prob [glioma]: 0.706567 --> Prob[me

Prob [meningoma]: 0.999999 --> Prob[meningoma]: 0.999998 
Prob [pituitary]: 0.984014 --> Prob[pituitary]: 0.664439 
Prob [meningoma]: 0.969381 --> Prob[meningoma]: 0.825722 
Prob [meningoma]: 1.000000 --> Prob[meningoma]: 1.000000 
Prob [glioma]: 0.993392 --> Prob[glioma]: 0.899944 
Prob [meningoma]: 0.961969 --> Prob[meningoma]: 0.572638 
Prob [glioma]: 0.987772 --> Prob[glioma]: 0.884384 
Prob [pituitary]: 0.936293 --> Prob[pituitary]: 0.557460 
Prob [pituitary]: 0.916039 --> Prob[glioma]: 0.553922 
Prob [glioma]: 0.999942 --> Prob[glioma]: 0.999828 
Prob [glioma]: 0.950936 --> Prob[meningoma]: 0.539023 
Prob [pituitary]: 0.995805 --> Prob[pituitary]: 0.987836 
Prob [glioma]: 0.966663 --> Prob[glioma]: 0.873110 
Prob [glioma]: 0.656800 --> Prob[meningoma]: 0.479408 


In [22]:
print(ct)

67


In [14]:
datagen = ImageDataGenerator(rescale = 1./255)
adv_set = datagen.flow_from_dataframe(X_val,
                                      directory = './outputs/',
                                      x_col = 'image',
                                      y_col = 'label',
                                      target_size = size,
                                      color_mode = 'grayscale',
                                      class_mode = 'categorical',
                                      batch_size = 1,
                                      shuffle = True,
                                      interpolation = 'bilinear')

Found 67 validated image filenames belonging to 3 classes.


D:\Softwares\anaconda\envs\tf-gpu\lib\site-packages\keras_preprocessing\image\dataframe_iterator.py:279: UserWarning: Found 240 invalid image filename(s) in x_col="image". These filename(s) will be ignored.
  warnings.warn(


In [15]:
predict = keras_model.evaluate(adv_set)

67/67 [==============================] - 40s 26ms/step - loss: 0.7689 - acc: 0.3582


In [20]:
path = './outputs/'
adv = importing_data(path)
adv_df = pd.DataFrame({'image':adv})
val_df = X_val[~X_val.image.isin(adv_df.image)]

In [22]:
val_set = datagen.flow_from_dataframe(val_df,
                                      directory = './brain_tumor_dataset/bt_images/',
                                      x_col = 'image',
                                      y_col = 'label',
                                      target_size = size,
                                      color_mode = 'grayscale',
                                      class_mode = 'categorical',
                                      batch_size = 10,
                                      shuffle = True,
                                      interpolation = 'bilinear')

Found 240 validated image filenames belonging to 3 classes.


In [23]:
predict = keras_model.evaluate(val_set)

24/24 [==============================] - 3s 102ms/step - loss: 0.1983 - acc: 0.9458


In [ ]:
es = EarlyStopping(monitor='val_loss', mode='min', patience=2, restore_best_weights=True, verbose=1)

checkpoint_cb = ModelCheckpoint("./models/abiwinanda_adversarial.h5", save_best_only=True)

reduce_lr = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.2, patience = 3, min_lr = 1e-5, mode = 'min', verbose=1)

history_adv = keras_model.fit(adv_set, validation_data = val_set, epochs= 50, batch_size = 10, callbacks=[es, checkpoint_cb, reduce_lr])   

In [30]:
class_names = {0: "glioma", 1: "meningoma", 2:"pituitary"}
d = 1
iters = 600
popsize = 10
model_path = "./models/abiwinanda_adversarial.h5"

In [30]:
import keras 
keras_model = keras.models.load_model(model_path)
keras_model.compile(optimizer = "Adam", loss='categorical_crossentropy', metrics =['acc'])

In [33]:
from scipy.optimize import differential_evolution

for filename in val:
    pred_adv = 0
    prob_adv = 0
    orig = cv2.imread(os.path.join("./brain_tumor_dataset/bt_images/",filename))
    orig = cv2.cvtColor(orig, cv2.COLOR_BGR2GRAY)
    orig = cv2.resize(orig, size)
    
    img = orig.copy()
    img = img.reshape(64,64,1)
    img = np.expand_dims(img,0)
    shape = img.shape
    
    
    prob_orig = keras_model.predict(preprocess(img[0]))
    pred_orig = np.argmax(prob_orig)

    bounds = [(0, shape[1]-1), (0, shape[2]-1), (0, 255)] * d
    result = differential_evolution(optimize, bounds, maxiter=iters, popsize=popsize, tol=1e-5, callback=callback)

    adv_img = perturb(result.x)

    prob = keras_model.predict(preprocess(adv_img[0]))
    print('Prob [%s]: %f --> Prob[%s]: %f ' %(class_names[pred_orig], prob_orig[0][pred_orig], class_names[pred_adv], prob_adv))

Prob [pituitary]: 0.954144 --> Prob[pituitary]: 0.825594 
Prob [glioma]: 0.860344 --> Prob[meningoma]: 0.527857 
Prob [pituitary]: 0.992554 --> Prob[pituitary]: 0.966793 
Prob [pituitary]: 0.987573 --> Prob[pituitary]: 0.969629 
Prob [pituitary]: 0.840629 --> Prob[meningoma]: 0.668600 
Prob [glioma]: 0.998155 --> Prob[glioma]: 0.992669 
Prob [meningoma]: 0.987989 --> Prob[meningoma]: 0.965925 
Prob [pituitary]: 0.895624 --> Prob[pituitary]: 0.689340 
Prob [glioma]: 0.827091 --> Prob[glioma]: 0.659291 
Prob [pituitary]: 0.973637 --> Prob[pituitary]: 0.883462 
Prob [meningoma]: 0.994640 --> Prob[meningoma]: 0.982730 
Prob [meningoma]: 0.997744 --> Prob[meningoma]: 0.991588 
Prob [glioma]: 0.998323 --> Prob[glioma]: 0.992649 
Prob [pituitary]: 0.926151 --> Prob[pituitary]: 0.816158 
Prob [pituitary]: 0.660326 --> Prob[meningoma]: 0.502183 
Prob [meningoma]: 0.999340 --> Prob[meningoma]: 0.995403 
Prob [meningoma]: 0.999907 --> Prob[meningoma]: 0.999879 
Prob [meningoma]: 0.999989 --> Prob

Prob [meningoma]: 0.999774 --> Prob[meningoma]: 0.998700 
Prob [meningoma]: 0.999947 --> Prob[meningoma]: 0.999725 
Prob [pituitary]: 0.835095 --> Prob[pituitary]: 0.630596 
Prob [meningoma]: 0.921044 --> Prob[meningoma]: 0.715552 
Prob [pituitary]: 0.613560 --> Prob[meningoma]: 0.593313 
Prob [meningoma]: 0.999051 --> Prob[meningoma]: 0.992165 
Prob [meningoma]: 0.970721 --> Prob[meningoma]: 0.875269 
Prob [pituitary]: 0.726084 --> Prob[meningoma]: 0.503587 
Prob [pituitary]: 0.819475 --> Prob[pituitary]: 0.605439 
Prob [pituitary]: 0.994566 --> Prob[pituitary]: 0.983162 
Prob [pituitary]: 0.883765 --> Prob[pituitary]: 0.622293 
Prob [glioma]: 0.992179 --> Prob[glioma]: 0.969264 
Prob [meningoma]: 0.989445 --> Prob[meningoma]: 0.926158 
Prob [glioma]: 0.947137 --> Prob[glioma]: 0.750268 
Prob [pituitary]: 0.993931 --> Prob[pituitary]: 0.981699 
Prob [meningoma]: 0.893733 --> Prob[meningoma]: 0.705196 
Prob [meningoma]: 0.556289 --> Prob[glioma]: 0.525149 
Prob [pituitary]: 0.992303 --

Prob [meningoma]: 0.996990 --> Prob[meningoma]: 0.987746 
Prob [meningoma]: 0.999898 --> Prob[meningoma]: 0.999563 
Prob [meningoma]: 0.999540 --> Prob[meningoma]: 0.996462 
Prob [pituitary]: 0.992167 --> Prob[pituitary]: 0.915147 
Prob [meningoma]: 0.684012 --> Prob[glioma]: 0.503247 
Prob [meningoma]: 1.000000 --> Prob[meningoma]: 1.000000 
Prob [glioma]: 0.977176 --> Prob[glioma]: 0.905313 
Prob [meningoma]: 0.971067 --> Prob[meningoma]: 0.852010 
Prob [glioma]: 0.999122 --> Prob[glioma]: 0.997490 
Prob [pituitary]: 0.413600 --> Prob[glioma]: 0.495037 
Prob [pituitary]: 0.962098 --> Prob[pituitary]: 0.879936 
Prob [glioma]: 0.992115 --> Prob[glioma]: 0.976914 
Prob [glioma]: 0.997297 --> Prob[glioma]: 0.990635 
Prob [pituitary]: 0.983962 --> Prob[pituitary]: 0.969411 
Prob [meningoma]: 0.536106 --> Prob[glioma]: 0.605986 
Prob [meningoma]: 0.902503 --> Prob[glioma]: 0.637599 


In [34]:
print(ct)

36


In [35]:
path = './outputs/'
adv = importing_data(path)
adv_df = pd.DataFrame({'image':adv})
val = X_val[~X_val.image.isin(adv_df.image)]

In [36]:
val_set = datagen.flow_from_dataframe(val,
                                      directory = './brain_tumor_dataset/bt_images/',
                                      x_col = 'image',
                                      y_col = 'label',
                                      target_size = size,
                                      color_mode = 'grayscale',
                                      class_mode = 'categorical',
                                      batch_size = 10,
                                      shuffle = True,
                                      interpolation = 'bilinear')
predict = keras_model.evaluate(val_set)

Found 240 validated image filenames belonging to 3 classes.
24/24 [==============================] - 7s 262ms/step - loss: 0.1805 - acc: 0.9375


In [ ]:
import math

def ssim(img1, img2):
    C1 = (0.01 * 255)**2
    C2 = (0.03 * 255)**2

    img1 = img1.astype(np.float64)
    img2 = img2.astype(np.float64)
    kernel = cv2.getGaussianKernel(11, 1.5)
    window = np.outer(kernel, kernel.transpose())

    mu1 = cv2.filter2D(img1, -1, window)[5:-5, 5:-5]  # valid
    mu2 = cv2.filter2D(img2, -1, window)[5:-5, 5:-5]
    mu1_sq = mu1**2
    mu2_sq = mu2**2
    mu1_mu2 = mu1 * mu2
    sigma1_sq = cv2.filter2D(img1**2, -1, window)[5:-5, 5:-5] - mu1_sq
    sigma2_sq = cv2.filter2D(img2**2, -1, window)[5:-5, 5:-5] - mu2_sq
    sigma12 = cv2.filter2D(img1 * img2, -1, window)[5:-5, 5:-5] - mu1_mu2

    ssim_map = ((2 * mu1_mu2 + C1) * (2 * sigma12 + C2)) / ((mu1_sq + mu2_sq + C1) *
                                                            (sigma1_sq + sigma2_sq + C2))
    return ssim_map.mean()

def calculate_ssim(img1, img2):
    if not img1.shape == img2.shape:
        raise ValueError('Input images must have the same dimensions.')
    if img1.ndim == 2:
        return ssim(img1, img2)
    elif img1.ndim == 3:
        if img1.shape[2] == 1:
            return ssim(np.squeeze(img1), np.squeeze(img2))

In [27]:
calculate_ssim(adv_img[0], orig)

0.9884681036584159

In [27]:
from scipy.optimize import differential_evolution
filename_orig = "./brain_tumor_dataset/bt_images/8.jpg"
filename_adv = "./outputs/46.jpg"
pred_adv = 0
prob_adv = 0

orig = cv2.imread(filename_orig)
orig = cv2.cvtColor(orig, cv2.COLOR_BGR2GRAY)
orig = cv2.resize(orig, size)
img = orig.copy()
img = img.reshape(64,64,1)
img = np.expand_dims(img,0)
shape = img.shape
prob_orig = keras_model.predict(preprocess(img[0]))
pred_orig = np.argmax(prob_orig)

orig = cv2.imread(filename_adv)
orig = cv2.cvtColor(orig, cv2.COLOR_BGR2GRAY)
orig = cv2.resize(orig, size)
img = orig.copy()
img = img.reshape(64,64,1)
img = np.expand_dims(img,0)
shape = img.shape
prob_adv = keras_model.predict(preprocess(img[0]))
pred_adv = np.argmax(prob_adv)

print('Prob [%s]: %f --> Prob[%s]: %f' %(class_names[pred_orig], prob_orig[0][pred_orig], class_names[pred_adv], prob_adv[0][pred_adv]))

Prob [glioma]: 0.972148 --> Prob[glioma]: 0.617895


In [33]:
pred_adv = 0
prob_adv = 0
d=5
orig = cv2.imread(os.path.join("./brain_tumor_dataset/bt_images/8.jpg"))
orig = cv2.cvtColor(orig, cv2.COLOR_BGR2GRAY)
orig = cv2.resize(orig, size)

img = orig.copy()
img = img.reshape(64,64,1)
img = np.expand_dims(img,0)
shape = img.shape


prob_orig = keras_model.predict(preprocess(img[0]))
pred_orig = np.argmax(prob_orig)

bounds = [(0, shape[1]-1), (0, shape[2]-1), (0, 255)] * d
result = differential_evolution(optimize, bounds, maxiter=iters, popsize=popsize, tol=1e-5, callback=callback)

adv_img = perturb(result.x)

prob = keras_model.predict(preprocess(adv_img[0]))
print('Prob [%s]: %f --> Prob[%s]: %f ' %(class_names[pred_orig], prob_orig[0][pred_orig], class_names[pred_adv], prob_adv))
cv2.imwrite('./output/8_5.jpg', adv_img[0])

Prob [glioma]: 0.972148 --> Prob[meningoma]: 0.507021 


True